In [10]:
!pip install scikit-mobility

     |████████████████████████████████| 154 kB 210 kB/s eta 0:00:01
     |████████████████████████████████| 9.9 MB 363 kB/s eta 0:00:01
     |████████████████████████████████| 1.1 MB 343 kB/s eta 0:00:01
     |████████████████████████████████| 95 kB 391 kB/s eta 0:00:01
     |████████████████████████████████| 994 kB 70 kB/s eta 0:00:013
     |████████████████████████████████| 78 kB 317 kB/s eta 0:00:01
  Using cached Jinja2-3.1.2-py3-none-any.whl (133 kB)
     |████████████████████████████████| 7.7 MB 1.1 MB/s eta 0:00:01
     |████████████████████████████████| 16.6 MB 1.9 MB/s eta 0:00:01
     |████████████████████████████████| 2.1 MB 674 kB/s eta 0:00:01
  Using cached attrs-21.4.0-py2.py3-none-any.whl (60 kB)
  Using cached click_plugins-1.1.1-py2.py3-none-any.whl (7.5 kB)
     |████████████████████████████████| 532 kB 1.0 MB/s eta 0:00:01
     |████████████████████████████████| 3.2 MB 1.2 MB/s eta 0:00:01
  Using cached texttable-1.6.4-py2.py3-none-any.whl (10 kB)
     |███████████

In [12]:
!python -m pip install "dask[complete]"

     |████████████████████████████████| 1.1 MB 609 kB/s eta 0:00:01
     |████████████████████████████████| 140 kB 1.0 MB/s eta 0:00:01
     |████████████████████████████████| 55 kB 865 kB/s eta 0:00:01
     |████████████████████████████████| 18.5 MB 11.8 MB/s eta 0:00:01
     |████████████████████████████████| 872 kB 3.3 MB/s eta 0:00:01
     |████████████████████████████████| 322 kB 5.3 MB/s eta 0:00:01
  Attempting uninstall: typing-extensions
    Found existing installation: typing-extensions 3.7.4.3
    Uninstalling typing-extensions-3.7.4.3:
      Successfully uninstalled typing-extensions-3.7.4.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.5.0 requires typing-extensions~=3.7.4, but you have typing-extensions 4.2.0 which is incompatible.


In [2]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))


from sklearn.cluster import KMeans
import numpy as np
import pandas as pd
import skmob
import dask.dataframe as dd
from dask.distributed import Client, LocalCluster

import math
from mpmath import * #contains the Lambert W function


In [3]:
cluster = LocalCluster()
client = Client(cluster)
client

/home/yakini/tool/anaconda3/envs/these/lib/python3.8/site-packages/distributed/node.py:177: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 39355 instead
  warnings.warn(


Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:39355/status,
Dashboard: http://127.0.0.1:39355/status,Workers: 4
Total threads: 8,Total memory: 15.33 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:36117,Workers: 4
Dashboard: http://127.0.0.1:39355/status,Total threads: 8
Started: Just now,Total memory: 15.33 GiB
Comm: tcp://127.0.0.1:41501,Total threads: 2
Dashboard: http://127.0.0.1:37983/status,Memory: 3.83 GiB
Nanny: tcp://127.0.0.1:41901,


In [4]:
# Chargement sous forme de dask dataframe. Adresse du fichier à changer.
dd_traces = dd.read_csv(r"../INSAnonym/dataset.csv", sep = '\t', header = None, \
                        names = ['ID', 'DateTime', 'lat', 'lon']).rename(columns = {'lat':'lon', 'lon':'lat'})

In [5]:
# Fenêtres de 15min : fait sur le dask dataframe (parallélisation)
agr = 15 # Temps en secondes
freq_str = f'{15}min'
truncated_datetime_col = f'DateTime{freq_str}'

dd_traces[truncated_datetime_col] = dd_traces.DateTime.astype(np.datetime64).dt.floor(str(agr)+"T")

# Dataset avec les positions par tranche de 15min : à faire sur le pandas dataframe pour avoir la median (fonction d'aggrégation non disponible dans Dask)
# dd_traces15mn = dd_traces[['ID', truncated_datetime_col, 'lat', 'lon']].groupby(['ID', truncated_datetime_col]).mean().reset_index() # Moyenne bof en général, il vaut mieux mediane

In [ ]:
# Nécessaire pour passer de dask dataframe à pandas dataframe
df_traces = dd_traces.compute() # C'est ce qui est  en temps
dd_traces

In [6]:
# Groupby/apply pour éviter l'itération
df_traces15mn = df_traces[['ID', truncated_datetime_col, 'lat', 'lon']].groupby(['ID', truncated_datetime_col]).median().reset_index()

In [7]:
import datetime

# Interpolation pour les trous de datetime : pour les bords, on répète la localisation de départ ou celle d'arrivée
def interpolate_15min(trace):

    first_line = pd.DataFrame({'DateTime15min':[datetime.datetime.combine(trace.DateTime15min.iloc[0].date(), datetime.time(0))], \
                                   'lat':[trace.lat.iloc[0]], 'lon':[trace.lon.iloc[0]]})

    last_line = pd.DataFrame({'DateTime15min':[datetime.datetime.combine(trace.DateTime15min.iloc[-1].date(), datetime.time(23, 45))], \
                                   'lat':[trace.lat.iloc[-1]], 'lon':[trace.lon.iloc[-1]]})

    trace = pd.concat([first_line, trace])
    trace = pd.concat([trace, last_line])
    #trace = trace.reset_index(drop = True)
    trace = trace.fillna(method = 'ffill').fillna(method = 'bfill')
    trace = trace.drop_duplicates(['ID', 'DateTime15min'])

    return trace[['DateTime15min', 'lat', 'lon']].set_index('DateTime15min').resample('15min').interpolate(method = 'linear').reset_index()


df_traces15mn_interpolated = df_traces15mn.groupby('ID').apply(interpolate_15min).reset_index().drop(columns = ['level_1'])
#df_traces15mn_interpolated.to_pickle("df_traces15mn_interpolated.pkl")




In [8]:
df_traces15mn_interpolated

,ID,DateTime15min,lat,lon
0,1,2015-03-04 00:00:00,45.772198,4.870504
1,1,2015-03-04 00:15:00,45.772198,4.870504
2,1,2015-03-04 00:30:00,45.772198,4.870504
3,1,2015-03-04 00:45:00,45.771409,4.871565
4,1,2015-03-04 01:00:00,45.771080,4.871303
...,...,...,...,...
304315,110,2015-03-12 22:45:00,48.891772,2.343127
304316,110,2015-03-12 23:00:00,48.891772,2.343127
304317,110,2015-03-12 23:15:00,48.891772,2.343127
304318,110,2015-03-12 23:30:00,48.891772,2.343127


In [9]:
test = df_traces15mn_interpolated[['ID', 'DateTime15min', 'lat', 'lon']]
tdf_traces_petit = skmob.TrajDataFrame(test, latitude='lat', longitude='lon', datetime='DateTime15min', user_id='ID').reset_index(drop = True)
tdf_traces_petit.datetime = tdf_traces_petit.datetime.astype(str)
m = tdf_traces_petit.plot_trajectory()

/home/couchot/.local/lib/python3.8/site-packages/skmob/utils/plot.py:133: UserWarning: Only the trajectories of the first 10 users will be plotted. Use the argument `max_users` to specify the desired number of users, or filter the TrajDataFrame.
  warnings.warn("Only the trajectories of the first 10 users will be plotted. Use the argument `max_users` to specify the desired number of users, or filter the TrajDataFrame.")
/home/couchot/.local/lib/python3.8/site-packages/skmob/utils/plot.py:158: UserWarning: If necessary, trajectories will be down-sampled to have at most `max_points` points. To avoid this, sepecify `max_points=None`.
  warnings.warn("If necessary, trajectories will be down-sampled to have at most `max_points` points. To avoid this, sepecify `max_points=None`.")


In [10]:
m

In [12]:
def clustering(df,nbclusterpj=6):
    dfp = df.copy()
    df_time = dfp[['DateTime15min']]

    df_time.drop_duplicates(subset=['DateTime15min'], inplace=True)
    df_time['Date'] =df_time['DateTime15min'].dt.date
    df_time.sort_values(by='DateTime15min', inplace=True)
    df_time['ClusteringOrder'] = df_time.index//(24*60/agr)
    df_time['TimeID'] = df_time.groupby(['Date']).DateTime15min.rank().astype(int)


    df_temp_traj = pd.merge(dfp, df_time, on="DateTime15min", how='inner').reset_index()

    df_temp = df_temp_traj.loc[df_temp_traj.ClusteringOrder==0]

    df_temp_traj_lat=df_temp[['ID','lat','TimeID']]
    df_temp_traj_lat=df_temp_traj_lat.pivot_table(index=['ID'],columns='TimeID',values='lat').reset_index()
    df_temp_traj_lat = df_temp_traj_lat.add_prefix('lat_')
    df_temp_traj_lat.rename(columns={'lat_ID':'ID'}, inplace=True)

    df_temp_traj_lon=df_temp[['ID','lon','TimeID']]
    df_temp_traj_lon=df_temp_traj_lon.pivot_table(index=['ID'],columns='TimeID',values='lon').reset_index()
    df_temp_traj_lon = df_temp_traj_lon.add_prefix('lon_')
    df_temp_traj_lon.rename(columns={'lon_ID':'ID'}, inplace=True)

    df_temp_traj_lon_lat = pd.merge(df_temp_traj_lat, df_temp_traj_lon, on="ID", how='inner')

    kmeans=KMeans(n_clusters=nbclusterpj).fit_predict(df_temp_traj_lon_lat)
    df_temp_traj_lon_lat['cluster_id'] = kmeans
    df_temp_traj_lon_lat['Date'] =df_temp.loc[0,'Date']

    for x in range(1,len(df_temp_traj.ClusteringOrder.unique())):

        df_temp = df_temp_traj.loc[df_temp_traj.ClusteringOrder==x].reset_index()

        df_temp_traj_lat=df_temp[['ID','lat','TimeID']]
        df_temp_traj_lat=df_temp_traj_lat.pivot_table(index=['ID'],columns='TimeID',values='lat').reset_index()
        df_temp_traj_lat = df_temp_traj_lat.add_prefix('lat_')
        df_temp_traj_lat.rename(columns={'lat_ID':'ID'}, inplace=True)

        df_temp_traj_lon=df_temp[['ID','lon','TimeID']]
        df_temp_traj_lon=df_temp_traj_lon.pivot_table(index=['ID'],columns='TimeID',values='lon').reset_index()
        df_temp_traj_lon = df_temp_traj_lon.add_prefix('lon_')
        df_temp_traj_lon.rename(columns={'lon_ID':'ID'}, inplace=True)

        df_temp_merge_traj = pd.merge(df_temp_traj_lat, df_temp_traj_lon, on='ID', how='inner')

        kmeans=KMeans(n_clusters=5).fit_predict(df_temp_merge_traj)
        df_temp_merge_traj['cluster_id'] = kmeans + df_temp_traj_lon_lat.cluster_id.max() + 1

        df_temp_merge_traj['Date'] = df_temp.loc[0,'Date']

        df_temp_traj_lon_lat = df_temp_traj_lon_lat.append(df_temp_merge_traj)


    df_temp_traj_lon_lat = df_temp_traj_lon_lat.reset_index()
    df_temp_traj_lon_lat=df_temp_traj_lon_lat.drop(columns=['index'])
    df_temp_traj_lon_lat = df_temp_traj_lon_lat.rename(index={'TimeID': 'index'})

    #calculate the centroid of ecah cluster
    df_clusters = df_temp_traj_lon_lat.drop(columns=['ID'])
    clusters = df_clusters.groupby(['cluster_id']).mean()
    clusters.reset_index(inplace=True)

    df_traces15mn_interpolated_temp = dfp.copy()
    df_traces15mn_interpolated_temp['Date'] =df_traces15mn_interpolated_temp['DateTime15min'].dt.date

    #%%

    df_temp_traj_lon_lat_temp = df_temp_traj_lon_lat[['ID','cluster_id', 'Date']]

    df_traces15mn_interpolated_temp = pd.merge(df_traces15mn_interpolated_temp, df_temp_traj_lon_lat_temp, on=["ID",'Date'], how='inner').reset_index()
    df_traces15mn_interpolated_temp = pd.merge(df_traces15mn_interpolated_temp, df_time, on=["DateTime15min",'Date'], how='inner').reset_index()

    #%%

    df_traces15mn_interpolated_temp.drop(columns={'level_0'}, inplace=True)
    df_traces15mn_interpolated_temp = pd.merge(df_traces15mn_interpolated_temp, clusters, on=['cluster_id'], how='inner').reset_index()
    df_traces15mn_interpolated_temp.drop(columns={'level_0','index'}, inplace=True)

    #%%

    for index, row in df_traces15mn_interpolated_temp.iterrows():
        column_lat_name='lat_'+str(df_traces15mn_interpolated_temp.loc[index,"TimeID"])
        column_lon_name='lon_'+str(df_traces15mn_interpolated_temp.loc[index,"TimeID"])
        df_traces15mn_interpolated_temp.loc[index,"clustered_lat"] = df_traces15mn_interpolated_temp.loc[index,column_lat_name]
        df_traces15mn_interpolated_temp.loc[index,"clustered_lon"] = df_traces15mn_interpolated_temp.loc[index,column_lon_name]

    #%%

    df_traces15mn_original_noisy =df_traces15mn_interpolated_temp[['ID','DateTime15min','Date','lat','lon','clustered_lat','clustered_lon']]

    df_clusters_num_ind=df_traces15mn_interpolated_temp[['cluster_id','ID']]
    df_clusters_num_ind.drop_duplicates(inplace=True)
    df_clusters_num_ind = df_clusters_num_ind.groupby(['cluster_id']).size().reset_index(name='counts')
    df_clusters_num_ind['Delete_cluster'] = pd.Series(df_clusters_num_ind['counts']<5)

    df_traces15mn_interpolated_temp2= df_traces15mn_interpolated_temp[['ID','DateTime15min','Date','lat','lon','clustered_lat','clustered_lon','cluster_id']]

    df_traces15mn_original_noisy_del = df_traces15mn_interpolated_temp2.join(df_clusters_num_ind.set_index('cluster_id'), on='cluster_id')
    df_traces15mn_original_noisy_del = df_traces15mn_original_noisy_del[['ID','DateTime15min','Date','lat','lon','clustered_lat','clustered_lon','Delete_cluster']]

    df_traces15mn_original_noisy_del.to_pickle("df_traces15mn_original_noisy_del.pkl")








    return df_traces15mn_original_noisy_del



In [13]:
df_traces15mn_interpolated_clustered = clustering(df_traces15mn_interpolated,5)
#df_traces15mn_interpolated_clustered = pd.read_pickle("../df_traces15mn_withNoisyLatLon.pkl")


<ipython-input-12-f87e85488f84>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_time.drop_duplicates(subset=['DateTime15min'], inplace=True)
<ipython-input-12-f87e85488f84>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_time['Date'] =df_time['DateTime15min'].dt.date
<ipython-input-12-f87e85488f84>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_time.sort_values(by='D

In [14]:
def associate_POI(df, params):
    global size, nbPOI, night_start, night_end, work_start, work_end, weekend_start, weekend_end,seed
    dateColumnName= 'DateTime15min'
    size = params['size']
    nbPOI = params['nbPOI']
    seed = params['seed']
    night_start = pd.to_datetime(params['night_start'], format="%H").strftime("%H:%M")
    night_end = pd.to_datetime(params['night_end'], format="%H").strftime("%H:%M")
    work_start = pd.to_datetime(params['work_start'], format="%H").strftime("%H:%M")
    work_end = pd.to_datetime(params['work_end'], format="%H").strftime("%H:%M")
    weekend_start = pd.to_datetime(params['weekend_start'], format="%H").strftime("%H:%M")
    weekend_end = pd.to_datetime(params['weekend_end'], format="%H").strftime("%H:%M")
    df_orig = df.copy()
    seed = 0
    np.random.seed(seed)

    # Need to set index as date to use between_time
    df_orig.index = pd.DatetimeIndex(df_orig[dateColumnName])

    # df.index = pd.DatetimeIndex(df[dateColumnName])

    def conditions(df):
        return [
            (df[dateColumnName].dt.dayofweek < 4) & (
                df.index.isin(df.between_time(night_start, night_end, include_start=False, include_end=False).index)),
            (df[dateColumnName].dt.dayofweek < 4) & (
                df.index.isin(df.between_time(work_start, work_end, include_start=False, include_end=False).index)),
            (df[dateColumnName].dt.dayofweek >= 4) & (df.index.isin(
                df.between_time(weekend_start, weekend_end, include_start=False, include_end=False).index))
        ]

    # Add metadata for days and drop non special rows
    values = ['NIGHT', 'WORK', 'WEEKEND']
    df_orig['time'] = np.select(conditions(df_orig), values, None)
    df_orig = df_orig.dropna(subset=['time'])
    df_orig = df_orig.astype({'lon': 'float64', 'lat': 'float64', 'ID': 'int64'})
    return df_orig




In [15]:
ddf_traces15mn_interpolated_poi = associate_POI(df_traces15mn_interpolated_clustered,{"size": 2, "nbPOI": 3, "night_start": 22, "night_end": 6, "work_start": 9, "work_end": 16,
               "weekend_start": 10, "weekend_end": 18, "seed":0})
ddf_traces15mn_interpolated_poi


,ID,DateTime15min,Date,lat,lon,clustered_lat,clustered_lon,Delete_cluster,time
DateTime15min,,,,,,,,,
2015-03-04 00:00:00,1,2015-03-04 00:00:00,2015-03-04,45.772198,4.870504,45.771152,4.892673,False,NIGHT
2015-03-04 00:00:00,2,2015-03-04 00:00:00,2015-03-04,45.786265,4.879282,45.771152,4.892673,False,NIGHT
2015-03-04 00:00:00,4,2015-03-04 00:00:00,2015-03-04,45.755808,4.866166,45.771152,4.892673,False,NIGHT
2015-03-04 00:00:00,6,2015-03-04 00:00:00,2015-03-04,45.632137,5.146270,45.771152,4.892673,False,NIGHT
2015-03-04 00:00:00,7,2015-03-04 00:00:00,2015-03-04,45.786242,4.879744,45.771152,4.892673,False,NIGHT
...,...,...,...,...,...,...,...,...,...
2015-05-12 23:15:00,73,2015-05-12 23:15:00,2015-05-12,45.738155,4.852832,45.747083,4.859102,True,NIGHT
2015-05-12 23:30:00,72,2015-05-12 23:30:00,2015-05-12,45.756012,4.865372,45.747083,4.859102,True,NIGHT
2015-05-12 23:30:00,73,2015-05-12 23:30:00,2015-05-12,45.738155,4.852832,45.747083,4.859102,True,NIGHT


In [16]:
"""
import itertools
def is_selected_(df,selectrate=1):
    def randomselect(l,selectrate):
        li = [np.random.random() < selectrate for _ in range(l)]
        return li
    df_orig = df.copy()
    #df['day'] = pd.Series([r.date() for r in  df['date']])
    df_orig['day'] = df_orig['DateTime15min'].dt.date
    test = df_orig.groupby(by=['ID','day','time']).apply(lambda x: randomselect(x.shape[0],selectrate)).reset_index(drop=True)#[1]
    b= [t for t in test]
    c = list(itertools.chain.from_iterable(b))
    df_orig['selected'] = c
    return df_orig

ddf_traces15mn_interpolated_poi_selected =is_selected_(ddf_traces15mn_interpolated_poi,1)
ddf_traces15mn_interpolated_poi_selected
"""

"\nimport itertools\ndef is_selected_(df,selectrate=1):\n    def randomselect(l,selectrate):\n        li = [np.random.random() < selectrate for _ in range(l)]\n        return li\n    df_orig = df.copy()\n    #df['day'] = pd.Series([r.date() for r in  df['date']])\n    df_orig['day'] = df_orig['DateTime15min'].dt.date\n    test = df_orig.groupby(by=['ID','day','time']).apply(lambda x: randomselect(x.shape[0],selectrate)).reset_index(drop=True)#[1]\n    b= [t for t in test]\n    c = list(itertools.chain.from_iterable(b))\n    df_orig['selected'] = c\n    return df_orig\n\nddf_traces15mn_interpolated_poi_selected =is_selected_(ddf_traces15mn_interpolated_poi,1)\nddf_traces15mn_interpolated_poi_selected\n"

In [17]:

ddf_traces15mn_interpolated_poi_selected = ddf_traces15mn_interpolated_poi.copy()

In [18]:
def shuffle_(df):
    df_orig = df.copy()
    #df['day'] = pd.Series([r.date() for r in  df['date']])
    df_orig['day'] = df_orig['DateTime15min'].dt.date
    #df_orig['date'] = pd.to_datetime(df_orig['date'], format='%Y-%m-%d %H:%M:%S')
    test = df_orig.groupby(by=['ID','day','time'])[['lat','lon']].apply(lambda x: x.sample(frac=1)).reset_index(drop=True)#[1]
    df_orig['shuff_lat'] = test.lat.values
    df_orig['shuff_lon'] = test.lon.values
    return df_orig

In [19]:
ddf_traces15mn_interpolated_poi_selected_shuffle =shuffle_(ddf_traces15mn_interpolated_poi_selected)
ddf_traces15mn_interpolated_poi_selected_shuffle

,ID,DateTime15min,Date,lat,lon,clustered_lat,clustered_lon,Delete_cluster,time,day,shuff_lat,shuff_lon
DateTime15min,,,,,,,,,,,,
2015-03-04 00:00:00,1,2015-03-04 00:00:00,2015-03-04,45.772198,4.870504,45.771152,4.892673,False,NIGHT,2015-03-04,45.772198,4.870504
2015-03-04 00:00:00,2,2015-03-04 00:00:00,2015-03-04,45.786265,4.879282,45.771152,4.892673,False,NIGHT,2015-03-04,45.770001,4.869676
2015-03-04 00:00:00,4,2015-03-04 00:00:00,2015-03-04,45.755808,4.866166,45.771152,4.892673,False,NIGHT,2015-03-04,45.770207,4.870024
2015-03-04 00:00:00,6,2015-03-04 00:00:00,2015-03-04,45.632137,5.146270,45.771152,4.892673,False,NIGHT,2015-03-04,45.770285,4.869538
2015-03-04 00:00:00,7,2015-03-04 00:00:00,2015-03-04,45.786242,4.879744,45.771152,4.892673,False,NIGHT,2015-03-04,45.770036,4.869665
...,...,...,...,...,...,...,...,...,...,...,...,...
2015-05-12 23:15:00,73,2015-05-12 23:15:00,2015-05-12,45.738155,4.852832,45.747083,4.859102,True,NIGHT,2015-05-12,48.891537,2.343147
2015-05-12 23:30:00,72,2015-05-12 23:30:00,2015-05-12,45.756012,4.865372,45.747083,4.859102,True,NIGHT,2015-05-12,48.891537,2.343147
2015-05-12 23:30:00,73,2015-05-12 23:30:00,2015-05-12,45.738155,4.852832,45.747083,4.859102,True,NIGHT,2015-05-12,48.891536,2.343146


In [20]:
import skmob
from skmob.measures.individual import radius_of_gyration

tdf_traces_petit = skmob.TrajDataFrame(ddf_traces15mn_interpolated_poi_selected_shuffle, latitude='lat', longitude='lon', datetime='DateTime15min', user_id='ID')

In [21]:
rg = radius_of_gyration(tdf_traces_petit)


100%|██████████| 69/69 [00:00<00:00, 150.03it/s]


In [22]:
rg


,uid,radius_of_gyration
0,1,2592.454493
1,2,108.231059
2,4,581.929523
3,5,158.764852
4,6,115.060888
...,...,...
64,87,30.389905
65,89,0.217711
66,98,94.707376
67,107,68.979758


In [23]:
ddf_traces15mn_interpolated_poi_selected_shuffle_gyr= ddf_traces15mn_interpolated_poi_selected_shuffle.join(rg.set_index('uid'), on='ID')
ddf_traces15mn_interpolated_poi_selected_shuffle_gyr.radius_of_gyration.max()

2778.6295478118595

In [24]:
def geoindis(df,epsilon=1,radius_of_gyration_selected=False):

    #
    # R = 200 #1km
    #l = np.log(3) #the lower the value the higher the privacy

    #epsilon = l/R

    df_orig = df.copy()
    earth_radius = 6378137
    global seed
    np.random.seed(seed)

    def chooseRadius(epsilon, z): # ok
        x = (z-1)/math.e
        return -(lambertw(x, k = -1)+1)/epsilon

    # http://movable-type.co.uk/scripts/latlong.html
    # Returns destination point given distance and bearing from start point
    def addVectorToPos(latitude, longitude, distance, angle):
        ang_distance = distance/earth_radius
        #lat1, lon1 = deg_to_cart(latitude, longitude)
        lat1 = math.radians(latitude)
        lon1 = math.radians(longitude)
        lat2 = math.asin(math.sin(lat1)*math.cos(ang_distance)+math.cos(lat1)*math.sin(ang_distance)*math.cos(angle))
        lon2 = lon1 + math.atan2(
            math.sin(angle)*math.sin(ang_distance)*math.cos(lat1),
            math.cos(ang_distance)-math.sin(lat1)*math.sin(lat2))
        lon2 = (lon2 + 3*math.pi)%(2*math.pi) - math.pi #normalise to -180,...,180
        return math.degrees(lat2), math.degrees(lon2), distance, angle


    def addNoise(loc,epsilon=1):
        (latitude, longitude) = loc
        theta = np.random.random()*math.pi*2
        z = np.random.random()
        r = chooseRadius(epsilon, z)
        return addVectorToPos(latitude, longitude, r, theta)



    def memoization(id,d_id_loc,loc,epsilon=1):
        r=loc
        def round_loc(loc,dec=2):
            lat,lon = loc
            return(round(lat,dec),round(lon,dec))
        if id not in d_id_loc:
            d_id_loc[id] = dict({})
        rloc = round_loc(loc)
        if rloc not in d_id_loc[id]:
            r= addNoise(loc,epsilon)[:2]
            d_id_loc[id][rloc] = r
        else :
            r= d_id_loc[id][rloc]
        return addNoise(r,epsilon)[:2]

    new_lats = []
    new_long = []
    d_id_loc=dict({})
    for index, row in df_orig.iterrows():
        if radius_of_gyration_selected :
            R = row['radius_of_gyration']
            l = np.log(3) #the lower the value the higher the privacy
            epsilon = min(epsilon, l/R)
        nla,nlo = memoization(row['ID'],d_id_loc,(row['lat'], row['lon']),epsilon)
        new_lats += [float(nla)]
        new_long += [float(nlo)]
    df_orig['geoinds_lat'] = np.array(new_lats, dtype=float)
    df_orig['geoindis_lon'] = np.array(new_long, dtype=float)
    return df_orig

ddf_traces15mn_interpolated_poi_selected_shuffle_geoindis =geoindis(ddf_traces15mn_interpolated_poi_selected_shuffle_gyr,0.0001,True)
ddf_traces15mn_interpolated_poi_selected_shuffle_geoindis


,ID,DateTime15min,Date,lat,lon,clustered_lat,clustered_lon,Delete_cluster,time,day,shuff_lat,shuff_lon,radius_of_gyration,geoinds_lat,geoindis_lon
DateTime15min,,,,,,,,,,,,,,,
2015-03-04 00:00:00,1,2015-03-04 00:00:00,2015-03-04,45.772198,4.870504,45.771152,4.892673,False,NIGHT,2015-03-04,45.772198,4.870504,2592.454493,45.425912,4.632550
2015-03-04 00:00:00,2,2015-03-04 00:00:00,2015-03-04,45.786265,4.879282,45.771152,4.892673,False,NIGHT,2015-03-04,45.770001,4.869676,108.231059,45.295942,5.194751
2015-03-04 00:00:00,4,2015-03-04 00:00:00,2015-03-04,45.755808,4.866166,45.771152,4.892673,False,NIGHT,2015-03-04,45.770207,4.870024,581.929523,45.913148,4.606333
2015-03-04 00:00:00,6,2015-03-04 00:00:00,2015-03-04,45.632137,5.146270,45.771152,4.892673,False,NIGHT,2015-03-04,45.770285,4.869538,115.060888,45.323527,4.947874
2015-03-04 00:00:00,7,2015-03-04 00:00:00,2015-03-04,45.786242,4.879744,45.771152,4.892673,False,NIGHT,2015-03-04,45.770036,4.869665,30.113446,46.129348,4.478864
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2015-05-12 23:15:00,73,2015-05-12 23:15:00,2015-05-12,45.738155,4.852832,45.747083,4.859102,True,NIGHT,2015-05-12,48.891537,2.343147,180.921024,45.557337,4.648068
2015-05-12 23:30:00,72,2015-05-12 23:30:00,2015-05-12,45.756012,4.865372,45.747083,4.859102,True,NIGHT,2015-05-12,48.891537,2.343147,187.863223,45.586662,5.084817
2015-05-12 23:30:00,73,2015-05-12 23:30:00,2015-05-12,45.738155,4.852832,45.747083,4.859102,True,NIGHT,2015-05-12,48.891536,2.343146,180.921024,45.268921,4.835539


In [25]:
ddf_traces15mn_interpolated_poi_selected_shuffle_geoindis.to_pickle("../data/annon.pkl")


In [26]:
test = ddf_traces15mn_interpolated_poi_selected_shuffle_geoindis[['ID', 'DateTime15min', 'geoinds_lat', 'geoindis_lon','Delete_cluster']]
test = test.loc[test['Delete_cluster'] == False]
tdf_traces_petit = skmob.TrajDataFrame(test, latitude='geoinds_lat', longitude='geoindis_lon', datetime='DateTime15min', user_id='ID').reset_index(drop = True)
tdf_traces_petit.datetime = tdf_traces_petit.datetime.astype(str)
mp = tdf_traces_petit.plot_trajectory()

/home/couchot/.local/lib/python3.8/site-packages/skmob/utils/plot.py:133: UserWarning: Only the trajectories of the first 10 users will be plotted. Use the argument `max_users` to specify the desired number of users, or filter the TrajDataFrame.
  warnings.warn("Only the trajectories of the first 10 users will be plotted. Use the argument `max_users` to specify the desired number of users, or filter the TrajDataFrame.")
/home/couchot/.local/lib/python3.8/site-packages/skmob/utils/plot.py:158: UserWarning: If necessary, trajectories will be down-sampled to have at most `max_points` points. To avoid this, sepecify `max_points=None`.
  warnings.warn("If necessary, trajectories will be down-sampled to have at most `max_points` points. To avoid this, sepecify `max_points=None`.")


In [27]:
mp